# Extraindo dados do site da CBF com Beautiful Soup

[Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/) é uma biblioteca de raspagem de dados _ ou coleta de dados se preferir _ que é uma forma de mineração que faz a extração de dados de páginas estruturadas da web. Ou seja, ao invés de entrar numa página, copiar o conteúdo, colar num bloco de dados, limpar os formatos, copiar cada item para sei lá, uma planilha, é possível desenvolver um código que faz tudo isso por você. 

No caso da biblioteca em questão, ela faz essa coleta a partir de páginas em HTML, por isso é preciso conhecer a linguagem _ além de manjar de CSS.

Nos links de referência, tem guias para saber mais sobre como funciona, mas aqui vamos a um exemplo de extração de dados da página da CBF.

**Exemplo**

Extrair os dados com todos os placares de jogos do Campeonato Brasileiro da Série B.

https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b

Perceba que na página, tem a tabela final do campeonato, mas na lateral, tem os resultados dos jogos da rodada 38 (a última).

<div style='text-align:center'>
    <img src='dados/tabela_serie_b_3.png' width=500 height=300 />
</div>


Ao analisar a estrutura do HTML da página (no modo desenvolvedor [F12] ou vendo o código fonte todo [ctrl+U]) é possível perceber que a região onde fica os dados de interesse, seguem mais ou menos nessa estrutura:

```html
<aside class="aside-rodadas swiper-container ... ">
    <div class="swiper-wrapper ... ">
        <div data-slide-index="0 ... 37" class="swiper-slide"> <!-- Isso se repete para cada rodada -->
            <header class="aside-header"> Rodada n </header>
            <div class="aside-content"> 
                <ul class="list-unstyled"> 
                    <li> Informações de cada jogo </li> <!-- Isso se repete para cada jogo -->
                 </ul>
            </div>
        </div>
    </div>
</aside>
```

Visualmente, quando clicamos nas setas que mostram cada rodada, sai um bloco e entra outro, mas olhando para o código, é possível ver que todas as infos estão lá disponíveis. É um plugin do javascript que cria esse efeito visual, de mostrar apenas um bloco por vez, e alterná-los ao gatilho dos botões de controle.

O que precisamos fazer, é baixar todo o HTML da página, e ir navegando através dele usando o beautiful soup para conseguirmos extrair o que precisamos.

**O que queremos?**

Vamos criar um dataframe com os seguintes dados:

- Ano
- Rodada
- Data do jogo
- Time da casa
- Gols do time da casa
- Time visitante
- Gols do time visitante


Feito isso, vamos salvar estes dados em CSV, para que não seja necessário repetir o processo toda vez.

Aliás, vamos fazer mais que isso: perceba que no site tem um combobox que permite ver os dados desde 2012 até o ano atual (estou escrevendo isso em maio de 2022, pode ser que algo mude), então vamos criar uma variável com o ano que queremos, e ai podemos baixar de acordo com a nossa necessidade.

### Bibliotecas usadas

Para baixar os dados da página, usaremos outra biblioteca, a `requests`, depois usaremos o `beautiful soup` para extrair os dados do HTML, e finalmente o `Pandas` para criarmos o dataframe e fazer a importação para o arquivo CSV. A biblioteca `re` que serve para lidar com expressões regulares, também será usada.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

### Coletar dados

In [3]:
# Vamos setar qual o ano queremos fazer a extração dos dados

ano = 2020

In [6]:
# URL de onde será extraído o HTML

url = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b/{}'.format(ano)

Para realizar a solicitação à página temos que informar ao site que somos um navegador e é para isso que usamos a variável headers

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

endereco_da_pagina = url

# no objeto_response iremos realizar o download da página da web 
objeto_response = requests.get(endereco_da_pagina, headers=headers)

# Se o status for OK, seguimos
if objeto_response.status_code != 200:
    print('Erro - Verifique se o link está correto ou se a página está fora do ar.')
else:
    print('Status code:', objeto_response.status_code)

200


Agora criaremos um objeto BeautifulSoup a partir do nosso objeto_response.

O parâmetro `html.parser` representa qual parser usaremos na criação do nosso objeto, um parser é um software responsável por realizar a conversão de uma entrada para uma estrutura de dados.

In [9]:
pagina_html = BeautifulSoup(objeto_response.content, 'html.parser')

Agora que temos uma variável contendo o HTML, podemos navegar por ele. Como visto na estrutura do código acima, temos que ir para o `aside.aside-rodadas` que é onde está contido os dados dos jogos.

In [10]:
aside_conteudo = pagina_html.find('aside', {'class': 'aside-rodadas'})

In [1]:
#print(aside_conteudo)

Uma dica para visualizar essa bagunça melhor, é ver pelo modo desenvolvedor do navegador, ou colar o código em uma IDE e identá-lo.

Agora que selecionamos apenas a parte que desejamos, temos uma estrutura em dois níveis:
    
- Cada uma das rodadas (`<div data-slide-index="0 ... 37"> ... <div class='aside-content'> ... `)
- Dentro das rodadas, cada um dos jogos (`<ul class='list-unstyled'><li> ... `)

Navegar por tags que possuem classes e ids é muito mais fácil, pois fica mais fácil exatamente quem queremos selecionar. Primeiro vamos pegar cada `div` onde estão contidos os códigos das rodadas, e atribuir como item em uma lista.

Depois da para imaginar o que será feito: será percorrida a lista com as rodadas, e dentro delas percorreremos todos os jogos, coletando tudo que for do nosso interesse.

In [44]:
anos = []
rodadas = []
datas_jogos = []
times_casa = []
times_visitante = []
placar_times_casa = []
placar_times_visitante = []

# Percorre cada rodada
for item in aside_conteudo.find('div').find_all('div', attrs={'class': ['swiper-slide']}):
    
    # obter as rodadas
    rodada = int(re.findall(r'\d+', item.h3.get_text())[0])
    
    for jogo in item.find_all('span', attrs={'class':['partida-desc']}): # Jogos
        
        # Data do jogo
        if(re.search(r'\d{2}/\d{2}/\d{4}', jogo.get_text())):
            mat = re.search(r'\d{2}/\d{2}/\d{4}', jogo.get_text())
            datas_jogos.append(mat.group())
        
    for jogo in item.find_all('div', attrs={'class':['clearfix']}): # Jogos
        
        # Time da casa
        div_casa = jogo.find("div", {'class': ['pull-left']})
        div_casa = div_casa.find("img", {"class": "icon"}, {"title":True})
        times_casa.append(div_casa['title'])
        
        # Time visitante
        div_visitante = jogo.find("div", {'class': ['pull-right']})
        div_visitante = div_visitante.find("img", {"class": "icon"}, {"title":True})
        times_visitante.append(div_visitante['title'])
        
        # Placar time da casa
        placar1 = jogo.find("strong", {'class': ['partida-horario']})
        txt = placar1.get_text().strip()
        placar_times_casa .append(int(txt.split('x')[0]))
        
        # Placar time visitante
        placar2 = jogo.find("strong", {'class': ['partida-horario']})
        txt = placar2.get_text().strip()
        placar_times_visitante .append(int(txt.split('x')[1]))
        
        # Rodada e Ano
        rodadas.append(rodada)
        anos.append(ano)

**Juntando tudo**

Percorrido o HTML e coletado os dados desejados, basta enviar tudo para um Dataframe.

In [48]:
df_serie_b = pd.DataFrame({
    'Ano': anos,
    'Rodada': rodadas, 
    'Data': datas_jogos,
    'Time casa': times_casa,
    'Gols time casa': placar_times_casa,
    'Time visitante': times_visitante,
    'Gols time visitante': placar_times_visitante
}).reset_index(drop=True) # o drop aqui, evita uma nova linha com os índices

In [49]:
df_serie_b

,Ano,Rodada,Data,Time casa,Gols time casa,Time visitante,Gols time visitante
0,2020,1,07/08/2020,Cuiabá - MT,0,Brasil - RS,0
1,2020,1,07/08/2020,Confiança - SE,2,Paraná - PR,2
2,2020,1,08/08/2020,Juventude - RS,2,Crb - AL,1
3,2020,1,08/08/2020,Operário - PR,3,Figueirense - SC,1
4,2020,1,08/08/2020,Avaí - SC,3,Náutico - PE,1
...,...,...,...,...,...,...,...
375,2020,38,29/01/2021,America - MG,2,Avaí - SC,1
376,2020,38,29/01/2021,Brasil - RS,0,Vitória - BA,1
377,2020,38,29/01/2021,Paraná - PR,0,Cruzeiro - MG,0
378,2020,38,29/01/2021,Guarani - SP,0,Juventude - RS,1


In [65]:
# Não temos dados ausentes, nem nulos

df_serie_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Ano                  380 non-null    int64 
 1   Rodada               380 non-null    int64 
 2   Data                 380 non-null    object
 3   Time casa            380 non-null    object
 4   Gols time casa       380 non-null    int64 
 5   Time visitante       380 non-null    object
 6   Gols time visitante  380 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 20.9+ KB


In [66]:
df_serie_b.describe()

,Ano,Rodada,Gols time casa,Gols time visitante
count,380.0,380.000000,380.000000,380.000000
mean,2020.0,19.500000,1.276316,0.892105
std,0.0,10.980313,1.104066,0.959024
min,2020.0,1.000000,0.000000,0.000000
25%,2020.0,10.000000,0.000000,0.000000
50%,2020.0,19.500000,1.000000,1.000000
75%,2020.0,29.000000,2.000000,1.000000
max,2020.0,38.000000,5.000000,7.000000


E voilá!!!

Bom, se tiver dúvidas, basta comparar o que tem no dataframe com o que tem no site, por exemplo escolhendo uma rodada aleatória.

In [55]:
df_serie_b.loc[df_serie_b['Rodada'] == 30]

,Ano,Rodada,Data,Time casa,Gols time casa,Time visitante,Gols time visitante
290,2020,30,18/12/2020,Oeste - SP,2,Vitória - BA,1
291,2020,30,18/12/2020,Juventude - RS,1,Csa - AL,0
292,2020,30,18/12/2020,Avaí - SC,1,Cruzeiro - MG,1
293,2020,30,18/12/2020,Cuiabá - MT,2,Operário - PR,0
294,2020,30,19/12/2020,Náutico - PE,1,Sampaio Correa - MA,0
295,2020,30,19/12/2020,Confiança - SE,1,Ponte Preta - SP,2
296,2020,30,20/12/2020,America - MG,2,Chapecoense - SC,2
297,2020,30,20/12/2020,Guarani - SP,2,Figueirense - SC,2
298,2020,30,20/12/2020,Crb - AL,1,Botafogo - SP,0
299,2020,30,21/12/2020,Paraná - PR,0,Brasil - RS,1


Não encontrei problemas. O número de linhas (os jogos) é exatamente o esperado. Como são 38 rodadas com 10 jogos cada, temos 380 jogos.

Finalmente, vamos salvar os dados num CSV, de forma que não seja necessário executar tudo isso de novo (ao menos que hajam alterações no projeto).

In [56]:
arquivo_dados = 'tabela_jogos_brasileiro_serie_b_{}.csv'.format(ano)

In [58]:
df_serie_b.to_csv(arquivo_dados, sep=';', encoding='utf-8', index=False)

**Melhorias**

Tem como ficar melhor?

Claro que sim! Poderíamos ter criado funções e classes para tratar das requests, das extrações e tratamentos de cada coluna, mas não era o objetivo aqui deste notebook.

Outra melhoria seria, antes de passar todo essa fase de coleta, verificar se existem os dados (se o csv está disponível) e se não existirem, aí partir para as extrações.

In [62]:
from os.path import exists

In [63]:
if exists(arquivo_dados):
    print('Arquivo existe')
    # ler csv
else:
    print('Arquivo não existe')
    # rodar scrapper

Arquivo existe


---

### Referências

https://beautiful-soup-4.readthedocs.io/en/latest/

https://medium.com/geekculture/web-scraping-tables-in-python-using-beautiful-soup-8bbc31c5803e

https://dev.to/lisandramelo/recebendo-informacoes-do-transfermarkt-uma-introducao-ao-web-scraping-188o

http://www.herongyang.com/XML/NPP-XML-Tools-Plugin.html
